In [20]:
import numpy as np
#import matplotlib.pylab as plt
#%matplotlib inline
import os,random
import cv2
#os.environ["KERAS_BACKEND"] = "theano"
#from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten#,MaxoutDense
from keras.layers.core import Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.datasets import mnist
import _pickle as cPickle, random, sys, keras
from keras.models import Model
from IPython import display
from keras.utils import np_utils
from tqdm import tqdm

# fix random seed for reproducibility
seed = 2000
np.random.seed(seed)

experiment='rv32by32_bs8'

In [23]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
def get_stats(X):
    print ('array shape:', X.shape)
    print ('min: %s  max: %s  mean: %s  std: %s' %(np.min(X),np.max(X),np.mean(X),np.std(X)))
    print ('-'*50)
    
def preprocess(X,Y,params):
    
    h=params['h']
    w=params['w']
    norm_type=params['norm_type']
    
    if Y is None:
        Y=np.zero_like(X)
        
    # downsample    
    X = cv2.resize(X, (w, h), interpolation=cv2.INTER_CUBIC)
    Y = cv2.resize(Y, (w, h), interpolation=cv2.INTER_CUBIC)
    
    # reshape        
    X = np.transpose(X,(2,0,1))
    X=np.expand_dims(X,axis=1)
    Y = np.transpose(Y,(2,0,1))
    Y=np.expand_dims(Y,axis=1)
    

    X = X.astype('float32')
    if norm_type is 'minusplusone':
        # normalized to [-1,1]
        xmaxd2=np.max(X)/2
        X-=xmaxd2
        X/=xmaxd2## load data
    elif norm_type is 'zeromeanunitvar':
        X-=np.mean(X)
        X/=np.std(X)
    else:
        X/=np.max(X)
    return X,Y

def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(10,8))
        plt.plot(losses["d"], label='discriminitive loss')
        plt.plot(losses["g"], label='generative loss')
        plt.legend()
        plt.show()
        
    
def plot_gen(n_ex=8,dim=(4,4), figsize=(10,10) ):
    noise = np.random.uniform(0,1,size=[n_ex,100])
    generated_images = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],2*i+1)
        img = generated_images[i,0,:,:]
        label = generated_images[i,1,:,:]>.5
        plt.imshow(img,cmap='Greys_r')
        plt.axis('off')
        plt.subplot(dim[0],dim[1],2*i+2)
        plt.imshow(label,cmap='Greys_r')
        
    plt.tight_layout()
    plt.show()
    
def train_for_n(nb_epoch=5000, plt_frq=50,BATCH_SIZE=32):

    for e in tqdm(range(nb_epoch)):  
        
        # Make generative images
        image_batch = X_train[np.random.randint(0,X_train.shape[0],size=BATCH_SIZE),:,:,:]    
        noise_gen = np.random.uniform(-1,1,size=[BATCH_SIZE,100])
        generated_images = generator.predict(noise_gen)
        
        # Train discriminator on generated images
        X = np.concatenate((image_batch, generated_images))
        y = np.zeros([2*BATCH_SIZE,1])
        y[0:BATCH_SIZE,0] = 1
        #y[BATCH_SIZE:,0] = 1
        
        #make_trainable(discriminator,True)
        d_loss  = discriminator.train_on_batch(X,y)
        losses["d"].append(d_loss)
    
        # train Generator-Discriminator stack on input noise to non-generated output class
        noise_tr = np.random.uniform(-1,1,size=[BATCH_SIZE,100])
        y2 = np.zeros([BATCH_SIZE,1])
        y2[:,0] = 1
        
        #make_trainable(discriminator,False)
        g_loss = GAN.train_on_batch(noise_tr, y2 )
        losses["g"].append(g_loss)
        
        # Updates plots
        if e%plt_frq==plt_frq-1:
            plot_loss(losses)
            plot_gen()
            
def pick_large_contours(X,Y,threshold):
    k1=0
    for k in range(Y.shape[2]):
        area=np.count_nonzero(Y[:,:,k])
        if area>threshold:
            if k1==0:
                Xs=X[:,:,k]
                Ys=Y[:,:,k]
                Xs=np.expand_dims(Xs,axis=2)
                Ys=np.expand_dims(Ys,axis=2)
            else:
                x1=X[:,:,k]
                x1=np.expand_dims(x1,axis=2)                
                Xs=np.append(Xs,x1,axis=2)
                y1=Y[:,:,k]
                y1=np.expand_dims(y1,axis=2)
                Ys=np.append(Ys,y1,axis=2)
            k1=k1+1
    #print 'Xs:'  %Xs.shape      
    #print 'Ys:'  %Ys.shape      
    #print '-' *50
    return Xs,Ys
        
def initNormal(shape, name=None):
	return initializations.normal(shape, scale=0.02, name=name)    

def grays_to_RGB(img):
    # turn 2D grayscale image into grayscale RGB
    return np.dstack((img, img, img))


def image_with_mask(img, mask,color=(0,255,0)):
    #img=np.asarray(img,dtype='uint8')
    mask=np.asarray(mask,dtype='uint8') 
    if np.max(mask)==1:
        mask=mask*255

    # returns a copy of the image with edges of the mask added in red
    if len(img.shape)==2:	
        img_color = grays_to_RGB(img)
    else:
        img_color =img

    mask_edges = cv2.Canny(mask, 100, 200) > 0
    img_color[mask_edges, 0] = color[0]  # set channel 0 to bright red, green & blue channels to 0
    img_color[mask_edges, 1] = color[1]
    img_color[mask_edges, 2] = color[2]
    img_color=img_color#/float(np.max(img))
    plt.axis('off')
    return img_color


def plot_real(n_ex=8,dim=(4,4), figsize=(10,10) ):
    
    idx = np.random.randint(0,X_train.shape[0],n_ex)
    generated_images = X_train[idx,:,:,:]

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],2*i+1)
        img = generated_images[i,0,:,:]
        plt.imshow(img,cmap='Greys_r')

        plt.subplot(dim[0],dim[1],2*i+2)
        label = generated_images[i,1,:,:]
        plt.imshow(label,cmap='Greys_r')
        
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [102]:
##LOAD DATA

path2data='.\\data\\test1\\train_HR\\'
h, w = 512, 512

train_list = []
print(path2data)
for file in os.listdir(path2data):
    img_test = cv2.imread(path2data+file)
    print(img_test.shape)
    train_list.append(img_test)


print(train_list[0].shape)
#X_train = train_list

X_train = []
for item in train_list:
    print(item[0].shape)
    X_train.append((item[0],item[1],item[2]))

X_train = np.array(X_train)
print(X_train.shape)
"""
path2data='.\\data\\test1\\train_label\\'

image_list = []
print(path2data)
for file in os.listdir(path2data):
    img_test = cv2.imread(path2data+file)
    image_list.append(img_test)

Y_train = image_list



path2data='.\\data\\test1\\test_HR\\'
image_list = []
print(path2data)
for file in os.listdir(path2data):
    img_test = cv2.imread(path2data+file)
    image_list.append(img_test)

print(len(image_list))
#X_train = np.append(X_train,image_list)#,axis=1)
X_train.append(image_list)

path2data='.\\data\\test1\\test_label\\'
image_list = []
print(path2data)
for file in os.listdir(path2data):
    img_test = cv2.imread(path2data+file)
    image_list.append(img_test)

Y_train = np.append(Y_train,image_list)#,axis=2)"""

.\data\test1\train_HR\
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(512, 3)
(25, 3, 512, 3)


"\npath2data='.\\data\\test1\\train_label\\'\n\nimage_list = []\nprint(path2data)\nfor file in os.listdir(path2data):\n    img_test = cv2.imread(path2data+file)\n    image_list.append(img_test)\n\nY_train = image_list\n\n\n\npath2data='.\\data\\test1\\test_HR\\'\nimage_list = []\nprint(path2data)\nfor file in os.listdir(path2data):\n    img_test = cv2.imread(path2data+file)\n    image_list.append(img_test)\n\nprint(len(image_list))\n#X_train = np.append(X_train,image_list)#,axis=1)\nX_train.append(image_list)\n\npath2data='.\\data\\test1\\test_label\\'\nimage_list = []\nprint(path2data)\nfor file in os.listdir(path2data):\n    img_test = cv2.imread(path2data+file)\n    image_list.append(img_test)\n\nY_train = np.append(Y_train,image_list)#,axis=2)"

In [66]:
print(len(X_train))

23592960
